## What are Prototypes?

In JavaScript, every object has a hidden link to another object called its **prototype**. When you try to access a property on an object, JavaScript first looks at the object itself. If it doesn't find it, it looks at the object's prototype, then the prototype's prototype, and so on - forming a **prototype chain**.

Think of it like inheritance in a family tree: if you don't have something, you can borrow it from your parent, and if they don't have it, from their parent, etc.

We can explore this with ```Object.getPrototypeOf(obj)```

In [116]:
// Every object has a prototype

let obj = { a: 1 };
console.log(Object.getPrototypeOf(obj));  // Object.prototype

// Arrays have Array.prototype
let arrr = [1, 2, 3];
console.log(Object.getPrototypeOf(arrr));  // Array.prototype

// Functions have Function.prototype
function fn() {}
console.log(Object.getPrototypeOf(fn));   // Function.prototype

[Object: null prototype] { foo: "bar" }
Object(0) [ oddCnt: [Function (anonymous)] ]
{}


# Prototypes and Inheritance in JavaScript

Understanding JavaScript's prototype-based inheritance model.

## Object.create() Examples

Before diving into prototypes, let's explore `Object.create()` and prototype manipulation - fundamental concepts for understanding JavaScript's inheritance model.

### Basic Prototype Manipulation

**Note:** In Deno (which this notebook uses), directly assigning to `__proto__` doesn't work as expected. The proper way to set an object's prototype is using `Object.setPrototypeOf()`, which works consistently in both Node.js and Deno.

In [117]:
let obj = { cook: "Baba", meal: "Salad" };
let othObj = { value: 10 };

// Proper way to set prototype (works in both Node.js and Deno)
Object.setPrototypeOf(othObj, obj);

console.log(othObj.value);  // 10 (own property)
console.log(othObj.cook);   // "Baba" (inherited from obj)
console.log(othObj.meal);   // "Salad" (inherited from obj)

// Note: In Node.js, othObj.__proto__ = obj also works, but it's deprecated
// and doesn't work in Deno's strict mode

10
Baba
Salad


### Extending Built-in Prototypes (Use with Caution!)

In [118]:
// Adding a method to Array.prototype
Array.prototype.oddCnt = function () {
    return this.filter( el => el % 2).length;
};

console.log([].oddCnt.call([1, 2, 3, 5, 6, 7]));  // 4 (four odd numbers)

4


### Object.create() with Property Descriptors

In [119]:
let grandp = {
    name: "Baba",
    age: 105,
};

// Create an object with grandp as prototype and define properties
const parent = Object.create(grandp, {
    name: {
        writable: true,
        configurable: true,
        value: "Kitty",
    },
});

console.log(parent.name);  // "Kitty" (own property)
console.log(parent.age);   // 105 (inherited from grandparent)

Kitty
105


### Object.create() with Data and Accessor Properties

In [120]:
let o = Object.create(Object.prototype, {
    
    foo: {      // foo is a regular data property
        writable: true,
        configurable: true,
        value: "hello",
    },

    bar: {      // bar is an accessor property
        configurable: false,
        get() { // <<------  it has getter
            return 10;
        },
        set(value) { // ...  and setter
            console.log("Setting `o.bar` to", value);
        },
    },
});

console.log(o.foo);  // "hello"
console.log(o.bar);  // 10
o.bar = 42;          // "Setting `o.bar` to 42"

hello
10
Setting `o.bar` to 42


42

### Inspecting and Modifying Property Descriptors

JavaScript provides methods to inspect and modify property characteristics (descriptors) of objects.

In [ ]:
// Object.getOwnPropertyDescriptors() - see ALL property descriptors
let objX = { x: 10, y: 20, z: 33, value: "yes" };
objX.arr = [1, 23];

console.log("All descriptors:");
console.log(Object.getOwnPropertyDescriptors(objX));
// {
//   x: { value: 10, writable: true, enumerable: true, configurable: true },
//   y: { value: 20, writable: true, enumerable: true, configurable: true },
//   z: { value: 33, writable: true, enumerable: true, configurable: true },
//   value: { value: "yes", writable: true, enumerable: true, configurable: true },
//   arr: { value: [ 1, 23 ], writable: true, enumerable: true, configurable: true }
// }

In [ ]:
// Object.getOwnPropertyDescriptor() - get descriptor for ONE property
console.log("Descriptor for 'z':");
console.log(Object.getOwnPropertyDescriptor(objX, "z"));
// { value: 33, writable: true, enumerable: true, configurable: true }

In [ ]:
// Object.getOwnPropertyNames() - returns ALL own properties (including non-enumerable)
console.log("All property names:", Object.getOwnPropertyNames(objX));
// [ "x", "y", "z", "value", "arr" ]

In [ ]:
// Object.defineProperty() - modify property descriptor
// Making property 'z' non-enumerable (hidden from for...in loops)
Object.defineProperty(objX, "z", { enumerable: false, value: objX.z });

console.log("After making 'z' non-enumerable:");
console.log(objX);  // z is hidden from normal listing
// { x: 10, y: 20, value: "yes", arr: [ 1, 23 ] }

console.log("But 'z' still exists:");
console.log(objX.z);  // 33

In [ ]:
// Making property read-only (non-writable)
Object.defineProperty(objX, "x", { writable: false, value: objX.x });

console.log("After making 'x' read-only:");
try {
  objX.x = 499;  // throws error in strict mode
} catch (e) {
  console.log("Error:", e.message);
  // Error: Cannot assign to read only property 'x' of object
}

### Constructor Function that Works With or Without 'new'

In [121]:

function Cons() { // This function supports constructing both 
    let self;     // with new context and without, though not recommended

    if (this === global || this === undefined) {
        self = Object.create(Cons.prototype);
    } else {
        self = this;
    }

    self.baba = 10;
    return self;
}

let obj1 = new Cons();
let obj2 = Cons();

console.log(obj1.baba);  // 10
console.log(obj2.baba);  // 10
console.log(obj1 instanceof Cons);  // true
console.log(obj2 instanceof Cons);  // true

10
10
true
true


### Inheritance Pattern using Object.create()

In [122]:
function BaseClass() {
    return this;
}

BaseClass.prototype.val = 10;

function InheriClass() {
    return this;
}

// Set up inheritance using Object.create()
// This allows the prototype chain to be followed up
// where this basically says something along the line
// InheriClass.prototype = { __proto__ : BaseClass.prototype }
InheriClass.prototype = Object.create(BaseClass.prototype);

let v = new InheriClass();
console.log(v.val);  // 10 (inherited from BaseClass)

10


## The Prototype Chain (ELI5)

Imagine you're looking for a toy:
1. First, you check your own toy box
2. If not there, you ask your older sibling
3. If they don't have it, they ask your parents
4. If parents don't have it, they ask grandparents
5. Eventually, you reach the end (null)

This is exactly how JavaScript's prototype chain works!

In [123]:
let grandparent = { surname: "Smith" };
let myparent = Object.create(grandparent);
myparent.job = "Engineer";
let child = Object.create(myparent);
child.name = "Alice";

console.log(child.name);     // "Alice" (own property)
console.log(child.job);      // "Engineer" (from myparent)
console.log(child.surname);  // "Smith" (from grandparent)

// The chain: child -> parent -> grandparent -> Object.prototype -> null

Alice
Engineer
Smith


## 3. Constructor Functions and Prototypes

In [124]:
function Person(name, age) {
    this.name = name;
    this.age = age;
}

// Add methods to prototype (shared by all instances)
Person.prototype.greet = function() {
    return `Hi, I'm ${this.name}`;
};

const alice = new Person("Lisa", 25);
const bob = new Person("Zugg", 30);

console.log(alice.greet());     // "Hi, I'm Lisa"
console.log(bob.greet());       // "Hi, I'm Zugg"

// Both share the same greet method
console.log(alice.greet === bob.greet);  // true

Hi, I'm Lisa
Hi, I'm Zugg
true


## 4. Understanding `__proto__` vs `prototype`

- `prototype`: Property on constructor functions, defines what instances will inherit
- `__proto__`: Property on objects, points to the actual prototype object

Think of it this way:
- `prototype` is the blueprint
- `__proto__` is the actual connection

In [125]:
function Dog(name) {
    this.name = name;
}

Dog.prototype.bark = function() {
    return "Woof!";
};

const rex = new Dog("Rex");

console.log(Dog.prototype);     // Dog.prototype is the blueprint

// rex.__proto__ basically points to Dog.prototype
// but can only be accessible diretly in NodeJS 
// where in Deno there is a restriction to touch it

console.log(Object.getPrototypeOf(rex) === Dog.prototype);  // true

// rex can use bark because of the prototype chain
console.log(rex.bark());  // "Woof!"

{ bark: [Function (anonymous)] }
true
Woof!


## 5. Object.create() and Prototype Manipulation

In [126]:
// Object.create() creates object with specified prototype
const animal = {
    eat:   function() {    return "Eating...";    }
};

const dog = Object.create(animal);
dog.bark = function() {    return "Woof!";        };

console.log(dog.bark());  // "Woof!" (own method)
console.log(dog.eat());   // "Eating..." (from prototype)

console.log(Object.getPrototypeOf(dog) === animal);  // true

Woof!
Eating...
true


## 6. Inheritance Patterns

In [127]:
// Parent constructor
function Animal(name) {
    this.name = name;
}

Animal.prototype.eat = function() {
    return this.name + ' is eating';
};

function Dog(name, breed) {   // Child constructor
    Animal.call(this, name);  // Call parent constructor
    this.breed = breed;       // but using Dog's context (this)
}

if ( false ) {   
    // some sources do inheritence like this
    // but neve fix the constructor which is going to
    // refer to the Animal function if we leave it as is
    Dog.prototype = Object.create(Animal.prototype);

    // tus we also need to fix the constructor reference, as
    // the newly created prototype object has no constructor prop 
    // and will resolve to the Animal.constructor prop
    Dog.prototype.constructor = Dog; 
} else {
    // alternatively we can just say which is also more apparent
    Object.setPrototypeOf(Dog.prototype, Animal.prototype);
}

Dog.prototype.bark = function() {
    return this.name + ' says Woof!';
};

const rex = new Dog("Rex", "German Shepherd");
console.log(rex.eat());   // "Rex is eating" (inherited)
console.log(rex.bark());  // "Rex says Woof!" (own method)
console.log(Object.getPrototypeOf(rex).constructor)

Rex is eating
Rex says Woof!
[Function: Dog]


## Visual Prototype Chain Example

```
r+--------------------+
|   rex (instance)   |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|   Dog.prototype    |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|  Animal.prototype  |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|  Object.prototype  |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|        null        |
+--------------------+
```

In [133]:
// Verify the chain
console.log(Object.getPrototypeOf(rex) === Dog.prototype);  // true
console.log(Object.getPrototypeOf(Dog.prototype) === Animal.prototype);  // true
console.log(Object.getPrototypeOf(Animal.prototype) === Object.prototype);  // true
console.log(Object.getPrototypeOf(Object.prototype) === null);  // true

true
true
true
true


## ES6 Classes (Syntactic Sugar)

ES6 classes are just syntactic sugar over prototypes - they work the same way under the hood!

In [134]:
class ModernAnimal {
    constructor(name) {
        this.name = name;
    }
    
    eat() {
        return `${this.name} is eating light`;
    }
}

class FancyDog extends ModernAnimal {
    constructor(name, breed) {
        super(name);          // Call parent constructor
        this.breed = breed;
    }
    
    bark() {
        return this.name + 'says Zap Zap!';
    }
}

const rex = new FancyDog( "Rex", "German Cybernaut");
console.log(rex.eat());       // "Rex is eating"
console.log(rex.bark());      // "Rex says Woof!"

// Still uses prototypes under the hood
console.log(Object.getPrototypeOf(rex) === FancyDog.prototype);  // true
console.log(rex.constructor, Object.getPrototypeOf(rex).constructor);

Rex is eating light
Rexsays Zap Zap!
true
[class FancyDog extends ModernAnimal] [class FancyDog extends ModernAnimal]


## Summary

- **Prototypes** enable object inheritance in JavaScript
- **Prototype chain** is how JavaScript looks up properties
- **Constructor functions** use `prototype` to share methods
- **`__proto__`** is the actual link, **`prototype`** is the blueprint
- **ES6 classes** are syntactic sugar over prototypes
- Understanding prototypes is key to mastering JavaScript!

## Gotchas & Best Practices

While prototypes are powerful, there are some common pitfalls and best practices to keep in mind.

### a) Never Modify `Object.prototype`

Modifying `Object.prototype` is considered bad practice because it affects all objects in your application. This can lead to unexpected behavior and conflicts with other libraries. This is often called 'prototype pollution'.

In [130]:
// Bad practice!
Object.prototype.foo = 'bar';

const myObj = {};
console.log(myObj.foo); // 'bar'

// This can break for...in loops
for (const key in myObj) {
    console.log(key); // 'foo'
}

bar
foo


### b) Resetting the `constructor` Property

When you manually set the prototype of a constructor function, you overwrite its `constructor` property. It's important to reset it to point back to the correct constructor function. This ensures that `instanceof` and other mechanisms that rely on the constructor property work correctly. Or use `Object.setPrototypeOf` instead.

In [131]:
function Animal() {}
function Dog() {}

Dog.prototype = Object.create(Animal.prototype);
// Without this next line, rex.constructor would be Animal
Dog.prototype.constructor = Dog;

// or use 
Object.setPrototypeOf(Dog.prototype, Animal.prototype);

const rex = new Dog();
console.log(rex.constructor === Dog); // true

true


##  Mixins

Mixins are a way to add functionality to objects or classes without using inheritance. A mixin is an object that contains methods to be used by other objects. You can use `Object.assign()` to copy the methods from the mixin to the prototype of a constructor function or class.

In [132]:
const flyMixin = {
  fly() {
    console.log(`${this.name} is flying!`);
  }
};

function Bird(name) {
  this.name = name;
}

Object.assign(Bird.prototype, flyMixin);

const eagle = new Bird('Eagle');
eagle.fly(); // Eagle is flying!

Eagle is flying!


## Exercises

### Exercise 1: Fix the Broken Prototype Chain

The following code has a broken prototype chain. Fix it so that `cat.purr()` and `cat.eat()` both work.

In [ ]:
function Animal() { }
Animal.prototype.munch = function() { console.log('munching...') };

function Cat() { }
// Fix the line below
Cat.prototype = new Animal();

Cat.prototype.purr = function() { console.log('purrrr') };

const cat = new Cat();
cat.purr();
cat.munch();

purrrr


TypeError: cat.eat is not a function

### Exercise 2: Implement a `SuperArray`

Create a `SuperArray` constructor that inherits from `Array`. Add a method to the `SuperArray` prototype called `last()` that returns the last element of the array.

In [ ]:
function SuperArray(...args) {
  const arr = new Array(...args);
  Object.setPrototypeOf(arr, SuperArray.prototype);
  return arr;
}

SuperArray.prototype = Object.create(Array.prototype);

// Add your method here

const myArr = new SuperArray(1, 2, 3, 4, 5);
console.log(myArr.last()); // 5

TypeError: myArr.last is not a function